임수진, 디지털 경영 트랙, 석사 1학기

참고자료
- 책 : 이것이 데이터 분석이다

# 인터파크 "관심집중 핫이슈" 크롤링

1. 내용
- 인터파크 페이지에서 뮤지컬, 콘서트, 연극 탭의 "관심집중 핫이슈" 작품만을 크롤링
- 인기가 많았던 작품뿐만 아니라 뜨고 있는 작품들을 한꺼번에 확인할 수 있는 페이지임

2. 데이터 상세 정보
- 크롤링 url
    - 뮤지컬 : http://ticket.interpark.com/MusicalIndex.asp
- 피처 설명
    - title : 작품 이름
    - category : 상세 정보
    - location : 장소
    - date : 시간
    - actor : 연극하는 사람

In [208]:
# -*- coding: utf-8 -*-

%matplotlib inline

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [209]:
musical_url = "http://ticket.interpark.com/MusicalIndex.asp"

req = requests.get(musical_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
musical_posters_list = soup.find_all(name="dt", attrs={"class":"issue_obj"})

musical_page_urls = []
for index in range(0, len(musical_posters_list)):
    a_url = musical_posters_list[index].find_all('a')
    href_url = a_url[0].get('href')
    musical_page_urls.append(href_url)

# 보여지는 작품 수가 계속 변경됨
count_musical_page_urls = len(musical_page_urls)
musical_page_urls

['http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002018',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=P0002351',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19019216',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20003125',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002020',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20001195',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002842',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20001382',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002972',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20000762',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19014431',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19009138',
 'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=16016299',

In [210]:
count_musical_page_urls

26

In [211]:
final_musical_page_url = []

for musical_page_url in musical_page_urls:
    req = requests.get(musical_page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    
    print(musical_page_url)

    try:
        title_category = soup.find(name="div", attrs={"class":"dt_Name"})
        title = title_category.find(name="span")   
        category = title_category.find(name="dd")
        final_musical_page_url.append(musical_page_url)
    except AttributeError:
        print("error")
        pass

http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002018
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=P0002351
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19019216
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20003125
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002020
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20001195
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002842
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20001382
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20002972
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=20000762
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19014431
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19009138
http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=16016299
http://ticket.interpark.com/Ticket/Goods/TPBridge.a

In [212]:
len(final_musical_page_url)

25

In [213]:
columns = ['title', 'category', 'location', 'date', 'actor']
df = pd.DataFrame(columns = columns)

for musical_page_url in final_musical_page_url:
    req = requests.get(musical_page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')

    title_category = soup.find(name="div", attrs={"class":"dt_Name"})
    title = title_category.find(name="span")
    category = title_category.find(name="dd")

    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""

    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
        
    basic_info = soup.find(name="ul", attrs={"class":"info_Lst"})
    basic_info = basic_info.text.strip()
    row_basic_info = list(basic_info.split('\n'))
    
    remove_word = ['', '관람시간 보기', '출연', '더보기']
    row_basic_info_final = []

    for item in row_basic_info:
        if item not in remove_word:
            row_basic_info_final.append(item)
            
    if row_basic_info_final[0][:2] == '부제':
        del row_basic_info_final[0]

        row_location = row_basic_info_final[0]
        if row_location[:2] == '장소':
            row_location = row_location[2:]

        row_date = row_basic_info_final[1]
        if row_date[:2] == '기간':
            row_date = row_date[2:]

        try:
            row_actor = row_basic_info_final[2]

        except IndexError:
            row_actor = ""

        row = [row_title, row_category, row_location, row_date, row_actor]
        series = pd.Series(row, index = df.columns)
        df = df.append(series, ignore_index=True)
        
    else:
        row_location = row_basic_info_final[0]
        if row_location[:2] == '장소':
            row_location = row_location[2:]

        row_date = row_basic_info_final[1]
        if row_date[:2] == '기간':
            row_date = row_date[2:]

        try:
            row_actor = row_basic_info_final[2]

        except IndexError:
            row_actor = ""

        row = [row_title, row_category, row_location, row_date, row_actor]
        series = pd.Series(row, index = df.columns)
        df = df.append(series, ignore_index=True)
        
df

,title,category,location,date,actor
0,뮤지컬〈아티스 ARTIS〉,뮤지컬 | 110분 | 만 13세이상,대학로예술극장 소극장,2020.03.21 ~ 2020.03.29,"김도빈,김히어라,안창용,현석준.."
1,뮤지컬 〈사운드 오브 뮤직〉,뮤지컬 | 140분 (인터미션 : 20분) | 만 7세이상,세종문화회관 대극장,2020.04.28 ~ 2020.05.17,"이연경,배다해,송일국,박성훈,서유나,서유진,허도영,임승연,권명현,우현아,한우종,유다..."
2,뮤지컬 〈미드나잇 : 앤틀러스〉,뮤지컬 | 90분 | 14세 이상 관람가,대학로 아트원씨어터 2관,2020.02.11 ~ 2020.05.03,"고상호,박은석,유리아,백형훈,안창용,황민수,김금나,김리.."
3,뮤지컬 〈미드나잇 : 액터뮤지션〉,뮤지컬 | 100분 | 14세 이상 관람가,예스24스테이지 3관,2020.04.11 ~ 2020.06.28,"고상호,신성민,조환지,김지철 (김영철),손유동,윤석현,김리,최연우,홍지희.."
4,뮤지컬 〈라흐마니노프〉,뮤지컬 | 100분 | 8세이상 관람가능,예스24스테이지 1관,2020.03.14 ~ 2020.06.07,"박규원,이해준,정욱진,유성재,정민,임병근.."
5,트롯연가,뮤지컬 | 120분 (인터미션 : 20분) | 만 7세이상,올림픽공원 우리금융아트홀,2020.03.31 ~ 2020.05.10,
6,마산 3.15의거 60주년 기념 뮤지컬 〈삼월의 그들〉,뮤지컬 | 150분 (인터미션 : 15분) | 만 13세이상,아르코예술극장 대극장,2020.03.21 ~ 2020.03.25,"허민진,임강성,박준휘,방선혁,김주호,정의욱,전수미,김은주.."
7,뮤지컬 〈데미안〉,뮤지컬 | 90분 | 만 7세이상,대학로 유니플렉스 2관,2020.03.07 ~ 2020.04.26,"정인지,유승현,전성민(김유영),김바다,김현진,김주연.."
8,뮤지컬 〈올 아이즈 온 미 (All Eyez on Me)〉,뮤지컬 | 100분 | 만 8세이상,대학로 유니플렉스 1관,2020.04.10 ~ 2020.05.24,"소냐,문혜원,김용진,정인성,레디,윤비,서동진,이민재,염승윤,백승리,최희재,박온,양병..."
9,세븐 핑거스 〈여행자〉 - 서울,뮤지컬 | 90분 | 8세이상 관람가능,LG아트센터,2020.06.17 ~ 2020.06.20,


----------------------

### (제외) Actor 전처리를 위한 부분

In [187]:
df_actor_master = df['actor']
df_actor = df['actor']
df_actor = df_actor.str.split(pat = ",", expand=True)
df_actor = df_actor[[0, 1]]
df_actor

,0,1
0,김도빈,김히어라
1,이연경,배다해
2,고상호,박은석
3,고상호,신성민
4,박규원,이해준
5,,None
6,허민진,임강성
7,정인지,유승현
8,소냐,문혜원
9,,None


In [200]:
temp = []
temp_2 = []

for actor in df_actor[1]:
    temp.append(actor)
        
temp

['김히어라',
 '배다해',
 '박은석',
 '신성민',
 '이해준',
 '임강성',
 '유승현',
 '문혜원',
 '이예은',
 '김윤후',
 '금보미..',
 '박정민',
 '한상민',
 '정재환',
 '오선미',
 '장인혁',
 '홍성희']

In [193]:
temp = []
temp_2 = []

for actor in df_actor[1]:
    temp.append(actor)
    
for i in temp:
    if '..' in i:
#         print("exist!!!")
        i = i[:-2]
        temp.append(i)
    else:
        temp_2.append(i)
        
temp_2

['김히어라',
 '배다해',
 '박은석',
 '신성민',
 '이해준',
 '임강성',
 '유승현',
 '문혜원',
 '이예은',
 '김윤후',
 '박정민',
 '한상민',
 '정재환',
 '오선미',
 '장인혁',
 '홍성희',
 '금보미']

In [198]:
temp_1 = []

for actor in df_actor[0]:
    temp_1.append(actor)
    
temp_1

['김도빈',
 '이연경',
 '고상호',
 '고상호',
 '박규원',
 '허민진',
 '정인지',
 '소냐',
 '김재선',
 '민용진',
 '박일곤',
 '김지환',
 '이우진',
 '정승환',
 '김다솔',
 '신재열',
 '안소연(쇼코)']

In [199]:
result = temp_1 + temp_2
result

['김도빈',
 '이연경',
 '고상호',
 '고상호',
 '박규원',
 '허민진',
 '정인지',
 '소냐',
 '김재선',
 '민용진',
 '박일곤',
 '김지환',
 '이우진',
 '정승환',
 '김다솔',
 '신재열',
 '안소연(쇼코)',
 '김히어라',
 '배다해',
 '박은석',
 '신성민',
 '이해준',
 '임강성',
 '유승현',
 '문혜원',
 '이예은',
 '김윤후',
 '박정민',
 '한상민',
 '정재환',
 '오선미',
 '장인혁',
 '홍성희',
 '금보미']

In [207]:
result_df = pd.DataFrame(result, columns=['actor'])
result_df.to_csv("./interpark_musical_actor.csv", mode='w')